In [2]:
# 일부만 할당된 GPU 메모리를 실행하는 동안 필요한만큼 늘릴 수 있도록 설정함
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
tf.logging.set_verbosity(tf.logging.ERROR)

In [4]:
import tensorflow as tf
import random
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 10
batch_size = 100

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 784]) # 나중에 변수 만들어줄테니 일단 만들어줘! 이런느낌! (그래프만들어주는느낌?)
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

# 28*28 -> max_pool을 통과하면서 14*14
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # stddev=0.01 : W의 값을 정규화시키겠다는 의미!!
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding="SAME")
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# 14*14 -> max_pool을 통과하면서 7*7
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")
L2 = tf.nn.relu(L2)
#L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# 7*7 -> max_pool을 통과하면서 7*7
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding="SAME")
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 4*4*128])

# Final FC 7X7X64 inputs -> 10 outputs
W4 = tf.get_variable("W3", shape=[4*4*128,10],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))

# full connected 계산하라!!
logits = tf.matmul(L3_flat, W4)+b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.7} # placeholder에 집어넣겠다는 이야기
        c, _= sess.run([cost, optimizer], feed_dict=feed_dict) # _이라는 말은 optimizer값은 받지 않겠다는 의미
        avg_cost += c/total_batch
        
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))
print("Learning Finished")

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) # argmax : 그 중에 가장 큰거를 1로 하겠다는 의미!!
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={
    X: mnist.test.images, Y: mnist.test.labels, keep_prob:1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples -1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print("Prdiction: ", sess.run(tf.argmax(logits,1), feed_dict={X: mnist.test.images[r:r+1],keep_prob:1}))

sess.close()

"""
Learning started. It takes sometime.
Epoch: 0001 cost = 0.422174428
Epoch: 0002 cost = 0.119945871
Epoch: 0003 cost = 0.086864322
Epoch: 0004 cost = 0.072373135
Epoch: 0005 cost = 0.060486251
Epoch: 0006 cost = 0.053713067
Epoch: 0007 cost = 0.049879163
Epoch: 0008 cost = 0.043746095
Epoch: 0009 cost = 0.041191877
Epoch: 0010 cost = 0.036772956
Learning Finished
Accuracy: 0.9924
Label:  [8]
Prdiction:  [8]

Learning started. It takes sometime.
Epoch: 0001 cost = 0.425049393
Epoch: 0002 cost = 0.121121024
Epoch: 0003 cost = 0.086250178
Epoch: 0004 cost = 0.072766873
Epoch: 0005 cost = 0.060146760
Epoch: 0006 cost = 0.053912209
Epoch: 0007 cost = 0.049555542
Epoch: 0008 cost = 0.043783873
Epoch: 0009 cost = 0.041362841
Epoch: 0010 cost = 0.037269008
Learning Finished
Accuracy: 0.9938
Label:  [3]
Prdiction:  [3]
"""

Learning started. It takes sometime.
Epoch: 0001 cost = 0.425049393
Epoch: 0002 cost = 0.121121024
Epoch: 0003 cost = 0.086250178
Epoch: 0004 cost = 0.072766873
Epoch: 0005 cost = 0.060146760
Epoch: 0006 cost = 0.053912209
Epoch: 0007 cost = 0.049555542
Epoch: 0008 cost = 0.043783873
Epoch: 0009 cost = 0.041362841
Epoch: 0010 cost = 0.037269008
Learning Finished
Accuracy: 0.9938
Label:  [3]
Prdiction:  [3]


'\nLearning started. It takes sometime.\nEpoch: 0001 cost = 0.422174428\nEpoch: 0002 cost = 0.119945871\nEpoch: 0003 cost = 0.086864322\nEpoch: 0004 cost = 0.072373135\nEpoch: 0005 cost = 0.060486251\nEpoch: 0006 cost = 0.053713067\nEpoch: 0007 cost = 0.049879163\nEpoch: 0008 cost = 0.043746095\nEpoch: 0009 cost = 0.041191877\nEpoch: 0010 cost = 0.036772956\nLearning Finished\nAccuracy: 0.9924\nLabel:  [8]\nPrdiction:  [8]\n'

In [3]:
tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 784]) # 나중에 변수 만들어줄테니 일단 만들어줘! 이런느낌! (그래프만들어주는느낌?)
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

# 28*28 -> max_pool을 통과하면서 14*14
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # stddev=0.01 : W의 값을 정규화시키겠다는 의미!!
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding="SAME")
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# 14*14 -> max_pool을 통과하면서 7*7
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# 7*7 -> max_pool을 통과하면서 7*7
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding="SAME")
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 4*4*128])

# Final FC 7X7X64 inputs -> 10 outputs
W4 = tf.get_variable("W3", shape=[4*4*128,10],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))

# full connected 계산하라!!
logits = tf.matmul(L3_flat, W4)+b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.5} # placeholder에 집어넣겠다는 이야기
        c, _= sess.run([cost, optimizer], feed_dict=feed_dict) # _이라는 말은 optimizer값은 받지 않겠다는 의미
        avg_cost += c/total_batch
        
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))
print("Learning Finished")

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) # argmax : 그 중에 가장 큰거를 1로 하겠다는 의미!!
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={
    X: mnist.test.images, Y: mnist.test.labels, keep_prob:1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples -1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print("Prdiction: ", sess.run(tf.argmax(logits,1), feed_dict={X: mnist.test.images[r:r+1],keep_prob:1}))

sess.close()

"""
Learning started. It takes sometime.
Epoch: 0001 cost = 0.510517319
Epoch: 0002 cost = 0.163072960
Epoch: 0003 cost = 0.125873183
Epoch: 0004 cost = 0.105937186
Epoch: 0005 cost = 0.091496085
Epoch: 0006 cost = 0.085221193
Epoch: 0007 cost = 0.078963502
Epoch: 0008 cost = 0.074712668
Epoch: 0009 cost = 0.070543856
Epoch: 0010 cost = 0.065415496
Epoch: 0011 cost = 0.065753135
Epoch: 0012 cost = 0.063677770
Epoch: 0013 cost = 0.059774008
Epoch: 0014 cost = 0.057365185
Epoch: 0015 cost = 0.057471330
Learning Finished
Accuracy: 0.9929
Label:  [3]
Prdiction:  [3]

Learning started. It takes sometime.
Epoch: 0001 cost = 0.510641918
Epoch: 0002 cost = 0.163007946
Epoch: 0003 cost = 0.126565942
Epoch: 0004 cost = 0.105909381
Epoch: 0005 cost = 0.091670244
Epoch: 0006 cost = 0.084679525
Epoch: 0007 cost = 0.079800526
Epoch: 0008 cost = 0.074142539
Epoch: 0009 cost = 0.070496633
Epoch: 0010 cost = 0.066021644
Epoch: 0011 cost = 0.065479567
Epoch: 0012 cost = 0.064372629
Epoch: 0013 cost = 0.059805956
Epoch: 0014 cost = 0.057697030
Epoch: 0015 cost = 0.057246343
Learning Finished
Accuracy: 0.9928
Label:  [7]
Prdiction:  [7]
"""

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Learning started. It takes sometime.
Epoch: 0001 cost = 0.510641918
Epoch: 0002 cost = 0.163007946
Epoch: 0003 cos

'\nLearning started. It takes sometime.\nEpoch: 0001 cost = 0.510517319\nEpoch: 0002 cost = 0.163072960\nEpoch: 0003 cost = 0.125873183\nEpoch: 0004 cost = 0.105937186\nEpoch: 0005 cost = 0.091496085\nEpoch: 0006 cost = 0.085221193\nEpoch: 0007 cost = 0.078963502\nEpoch: 0008 cost = 0.074712668\nEpoch: 0009 cost = 0.070543856\nEpoch: 0010 cost = 0.065415496\nEpoch: 0011 cost = 0.065753135\nEpoch: 0012 cost = 0.063677770\nEpoch: 0013 cost = 0.059774008\nEpoch: 0014 cost = 0.057365185\nEpoch: 0015 cost = 0.057471330\nLearning Finished\nAccuracy: 0.9929\nLabel:  [3]\nPrdiction:  [3]\n'

In [4]:
tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 784]) # 나중에 변수 만들어줄테니 일단 만들어줘! 이런느낌! (그래프만들어주는느낌?)
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

# 28*28 -> max_pool을 통과하면서 14*14
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # stddev=0.01 : W의 값을 정규화시키겠다는 의미!!
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding="SAME")
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# 14*14 -> max_pool을 통과하면서 7*7
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# 7*7 -> max_pool을 통과하면서 7*7
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding="SAME")
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 4*4*128])

# Final FC 7X7X64 inputs -> 10 outputs
W4 = tf.get_variable("W3", shape=[4*4*128,10],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))

# full connected 계산하라!!
logits = tf.matmul(L3_flat, W4)+b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.7} # placeholder에 집어넣겠다는 이야기
        c, _= sess.run([cost, optimizer], feed_dict=feed_dict) # _이라는 말은 optimizer값은 받지 않겠다는 의미
        avg_cost += c/total_batch
        
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))
print("Learning Finished")

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) # argmax : 그 중에 가장 큰거를 1로 하겠다는 의미!!
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={
    X: mnist.test.images, Y: mnist.test.labels, keep_prob:1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples -1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print("Prdiction: ", sess.run(tf.argmax(logits,1), feed_dict={X: mnist.test.images[r:r+1],keep_prob:1}))

sess.close()

"""
Learning started. It takes sometime.
Epoch: 0001 cost = 0.424806425
Epoch: 0002 cost = 0.120208484
Epoch: 0003 cost = 0.085613264
Epoch: 0004 cost = 0.071716400
Epoch: 0005 cost = 0.059897973
Epoch: 0006 cost = 0.053787526
Epoch: 0007 cost = 0.048903041
Epoch: 0008 cost = 0.044594258
Epoch: 0009 cost = 0.041729465
Epoch: 0010 cost = 0.036071912
Epoch: 0011 cost = 0.036255933
Epoch: 0012 cost = 0.034049949
Epoch: 0013 cost = 0.032128661
Epoch: 0014 cost = 0.030201911
Epoch: 0015 cost = 0.029466881
Learning Finished
Accuracy: 0.9939
Label:  [9]
Prdiction:  [9]

Learning started. It takes sometime.
Epoch: 0001 cost = 0.424698413
Epoch: 0002 cost = 0.120319841
Epoch: 0003 cost = 0.086474026
Epoch: 0004 cost = 0.072068285
Epoch: 0005 cost = 0.059969588
Epoch: 0006 cost = 0.054144932
Epoch: 0007 cost = 0.049947605
Epoch: 0008 cost = 0.044603801
Epoch: 0009 cost = 0.042135027
Epoch: 0010 cost = 0.038076872
Epoch: 0011 cost = 0.034954966
Epoch: 0012 cost = 0.033519498
Epoch: 0013 cost = 0.032702801
Epoch: 0014 cost = 0.030601980
Epoch: 0015 cost = 0.029379258
Learning Finished
Accuracy: 0.9931
Label:  [3]
Prdiction:  [3]
"""

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.424698413
Epoch: 0002 cost = 0.120319841
Epoch: 0003 cost = 0.086474026
Epoch: 0004 cost = 0.072068285
Epoch: 0005 cost = 0.059969588
Epoch: 0006 cost = 0.054144932
Epoch: 0007 cost = 0.049947605
Epoch: 0008 cost = 0.044603801
Epoch: 0009 cost = 0.042135027
Epoch: 0010 cost = 0.038076872
Epoch: 0011 cost = 0.034954966
Epoch: 0012 cost = 0.033519498
Epoch: 0013 cost = 0.032702801
Epoch: 0014 cost = 0.030601980
Epoch: 0015 cost = 0.029379258
Learning Finished
Accuracy: 0.9931
Label:  [3]
Prdiction:  [3]


'\nLearning started. It takes sometime.\nEpoch: 0001 cost = 0.424806425\nEpoch: 0002 cost = 0.120208484\nEpoch: 0003 cost = 0.085613264\nEpoch: 0004 cost = 0.071716400\nEpoch: 0005 cost = 0.059897973\nEpoch: 0006 cost = 0.053787526\nEpoch: 0007 cost = 0.048903041\nEpoch: 0008 cost = 0.044594258\nEpoch: 0009 cost = 0.041729465\nEpoch: 0010 cost = 0.036071912\nEpoch: 0011 cost = 0.036255933\nEpoch: 0012 cost = 0.034049949\nEpoch: 0013 cost = 0.032128661\nEpoch: 0014 cost = 0.030201911\nEpoch: 0015 cost = 0.029466881\nLearning Finished\nAccuracy: 0.9939\nLabel:  [9]\nPrdiction:  [9]\n\n\n'

In [4]:
tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 10
batch_size = 100

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 784]) # 나중에 변수 만들어줄테니 일단 만들어줘! 이런느낌! (그래프만들어주는느낌?)
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

# 28*28 -> max_pool을 통과하면서 14*14
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # stddev=0.01 : W의 값을 정규화시키겠다는 의미!!
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding="SAME")
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# 14*14 -> max_pool을 통과하면서 7*7
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# 7*7 -> max_pool을 통과하면서 7*7
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding="SAME")
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 4*4*128])

W4 = tf.Variable(tf.random_normal([3, 3, 128, 256], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding="SAME")
L4 = tf.nn.relu(L4)
L4 = tf.nn.max_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
L4_flat = tf.reshape(L4, [-1, 2*2*256])

# Final FC 7X7X64 inputs -> 10 outputs
W5 = tf.get_variable("W4", shape=[2*2*256,10],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))

# full connected 계산하라!!
logits = tf.matmul(L4_flat, W5)+b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.7} # placeholder에 집어넣겠다는 이야기
        c, _= sess.run([cost, optimizer], feed_dict=feed_dict) # _이라는 말은 optimizer값은 받지 않겠다는 의미
        avg_cost += c/total_batch
        
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))
print("Learning Finished")

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) # argmax : 그 중에 가장 큰거를 1로 하겠다는 의미!!
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={
    X: mnist.test.images, Y: mnist.test.labels, keep_prob:1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples -1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print("Prdiction: ", sess.run(tf.argmax(logits,1), feed_dict={X: mnist.test.images[r:r+1],keep_prob:1}))

sess.close()

"""
Learning started. It takes sometime.
Epoch: 0001 cost = 0.521165673
Epoch: 0002 cost = 0.115732721
Epoch: 0003 cost = 0.081753533
Epoch: 0004 cost = 0.066631483
Epoch: 0005 cost = 0.057669014
Epoch: 0006 cost = 0.052699069
Epoch: 0007 cost = 0.046082615
Epoch: 0008 cost = 0.043536871
Epoch: 0009 cost = 0.037597921
Epoch: 0010 cost = 0.036512583
Learning Finished
Accuracy: 0.9932
Label:  [2]
Prdiction:  [2]

Learning started. It takes sometime.
Epoch: 0001 cost = 0.520293894
Epoch: 0002 cost = 0.116156142
Epoch: 0003 cost = 0.081148065
Epoch: 0004 cost = 0.067255407
Epoch: 0005 cost = 0.059169368
Epoch: 0006 cost = 0.052693228
Epoch: 0007 cost = 0.046038137
Epoch: 0008 cost = 0.043084515
Epoch: 0009 cost = 0.037856678
Epoch: 0010 cost = 0.036542758
Learning Finished
Accuracy: 0.9938
Label:  [9]
Prdiction:  [9]
"""

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.520293894
Epoch: 0002 cost = 0.116156142
Epoch: 0003 cost = 0.081148065
Epoch: 0004 cost = 0.067255407
Epoch: 0005 cost = 0.059169368
Epoch: 0006 cost = 0.052693228
Epoch: 0007 cost = 0.046038137
Epoch: 0008 cost = 0.043084515
Epoch: 0009 cost = 0.037856678
Epoch: 0010 cost = 0.036542758
Learning Finished
Accuracy: 0.9938
Label:  [9]
Prdiction:  [9]


'\nLearning started. It takes sometime.\nEpoch: 0001 cost = 0.521165673\nEpoch: 0002 cost = 0.115732721\nEpoch: 0003 cost = 0.081753533\nEpoch: 0004 cost = 0.066631483\nEpoch: 0005 cost = 0.057669014\nEpoch: 0006 cost = 0.052699069\nEpoch: 0007 cost = 0.046082615\nEpoch: 0008 cost = 0.043536871\nEpoch: 0009 cost = 0.037597921\nEpoch: 0010 cost = 0.036512583\nLearning Finished\nAccuracy: 0.9932\nLabel:  [2]\nPrdiction:  [2]\n\n\n'

In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# 신경망 모델 구성
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

# 기본적으로 inputs, outputs size, kernel_size만 넣어주면
# 크게 신경쓰지 않아도 일반적으로 효율적인 신경망을 만들어줍니다.

L1 = tf.layers.conv2d(X, 32, [3, 3], activation=tf.nn.relu)
L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(L1, 64, [3, 3], activation=tf.nn.relu)
L2 = tf.layers.max_pooling2d(L1, [2,2], [2,2])
L2 = tf.layers.dropout(L1, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

model = tf.layers.dense(L3, 10, activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
# 신경망 모델 학습
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys, is_training: True})
        total_cost += cost_val
        
    print('Epoch : ' , '%04d'%(epoch+1),'Avg.cost =', '{:.4f}'.format(total_cost/total_batch))
    
print("최적화 완료!")

### 결과 확인 ###
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy, feed_dict = {X:mnist.test.images.reshape(-1, 28, 28, 1), 
                                              Y:mnist.test.labels, is_training: False}))

Epoch :  0001 Avg.cost = 0.1853
Epoch :  0002 Avg.cost = 0.0603
Epoch :  0003 Avg.cost = 0.0384
Epoch :  0004 Avg.cost = 0.0253
Epoch :  0005 Avg.cost = 0.0180
Epoch :  0006 Avg.cost = 0.0133
Epoch :  0007 Avg.cost = 0.0091
Epoch :  0008 Avg.cost = 0.0071
Epoch :  0009 Avg.cost = 0.0057
Epoch :  0010 Avg.cost = 0.0054
Epoch :  0011 Avg.cost = 0.0048
Epoch :  0012 Avg.cost = 0.0040
Epoch :  0013 Avg.cost = 0.0026
Epoch :  0014 Avg.cost = 0.0038
Epoch :  0015 Avg.cost = 0.0026
최적화 완료!
정확도: 0.9868


In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# 신경망 모델 구성
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

# 기본적으로 inputs, outputs size, kernel_size만 넣어주면
# 크게 신경쓰지 않아도 일반적으로 효율적인 신경망을 만들어줍니다.

L1 = tf.layers.conv2d(X, 32, [3, 3], activation=tf.nn.relu)
L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(L1, 64, [3, 3], activation=tf.nn.relu)
L2 = tf.layers.max_pooling2d(L1, [2,2], [2,2])
L2 = tf.layers.dropout(L1, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

model = tf.layers.dense(L3, 10, activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.0001).minimize(cost)

# 신경망 모델 학습
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(40):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys, is_training: True})
        total_cost += cost_val
        
    print('Epoch : ' , '%04d'%(epoch+1),'Avg.cost =', '{:.4f}'.format(total_cost/total_batch))
    
print("최적화 완료!")

### 결과 확인 ###
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy, feed_dict = {X:mnist.test.images.reshape(-1, 28, 28, 1), 
                                              Y:mnist.test.labels, is_training: False}))

In [4]:
tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 784]) # 나중에 변수 만들어줄테니 일단 만들어줘! 이런느낌! (그래프만들어주는느낌?)
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

# 28*28 -> max_pool을 통과하면서 14*14
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # stddev=0.01 : W의 값을 정규화시키겠다는 의미!!
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding="SAME")
L1 = tf.nn.leaky_relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# 14*14 -> max_pool을 통과하면서 7*7
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# 7*7 -> max_pool을 통과하면서 7*7
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding="SAME")
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 4*4*128])

W4 = tf.Variable(tf.random_normal([3, 3, 128, 256], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding="SAME")
L4 = tf.nn.relu(L4)
L4 = tf.nn.avg_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
L4_flat = tf.reshape(L4, [-1, 2*2*256])

# Final FC 7X7X64 inputs -> 10 outputs
W5 = tf.get_variable("W4", shape=[2*2*256,10],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))

# full connected 계산하라!!
logits = tf.matmul(L4_flat, W5)+b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.6} # placeholder에 집어넣겠다는 이야기
        c, _= sess.run([cost, optimizer], feed_dict=feed_dict) # _이라는 말은 optimizer값은 받지 않겠다는 의미
        avg_cost += c/total_batch
        
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))
print("Learning Finished")

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) # argmax : 그 중에 가장 큰거를 1로 하겠다는 의미!!
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={
    X: mnist.test.images, Y: mnist.test.labels, keep_prob:1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples -1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print("Prdiction: ", sess.run(tf.argmax(logits,1), feed_dict={X: mnist.test.images[r:r+1],keep_prob:1}))

sess.close()

"""
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.761663368
Epoch: 0002 cost = 0.163971262
Epoch: 0003 cost = 0.112958088
Epoch: 0004 cost = 0.089053273
Epoch: 0005 cost = 0.079811646
Epoch: 0006 cost = 0.069248633
Epoch: 0007 cost = 0.061179850
Epoch: 0008 cost = 0.057907291
Epoch: 0009 cost = 0.053819029
Epoch: 0010 cost = 0.050987588
Epoch: 0011 cost = 0.047628223
Epoch: 0012 cost = 0.043664427
Epoch: 0013 cost = 0.043872318
Epoch: 0014 cost = 0.042387685
Epoch: 0015 cost = 0.039887975
Learning Finished
Accuracy: 0.9944
Label:  [6]
Prdiction:  [6]

Learning started. It takes sometime.
Epoch: 0001 cost = 0.761674645
Epoch: 0002 cost = 0.164384049
Epoch: 0003 cost = 0.114068380
Epoch: 0004 cost = 0.089896361
Epoch: 0005 cost = 0.079109224
Epoch: 0006 cost = 0.069877380
Epoch: 0007 cost = 0.060602335
Epoch: 0008 cost = 0.057568864
Epoch: 0009 cost = 0.053724480
Epoch: 0010 cost = 0.051536912
Epoch: 0011 cost = 0.047666873
Epoch: 0012 cost = 0.043789683
Epoch: 0013 cost = 0.042038869
Epoch: 0014 cost = 0.041594663
Epoch: 0015 cost = 0.040478235
Learning Finished
Accuracy: 0.9933
Label:  [0]
Prdiction:  [0]
"""

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.761674645
Epoch: 0002 cost = 0.164384049
Epoch: 0003 cost = 0.114068380
Epoch: 0004 cost = 0.089896361
Epoch: 0005 cost = 0.079109224
Epoch: 0006 cost = 0.069877380
Epoch: 0007 cost = 0.060602335
Epoch: 0008 cost = 0.057568864
Epoch: 0009 cost = 0.053724480
Epoch: 0010 cost = 0.051536912
Epoch: 0011 cost = 0.047666873
Epoch: 0012 cost = 0.043789683
Epoch: 0013 cost = 0.042038869
Epoch: 0014 cost = 0.041594663
Epoch: 0015 cost = 0.040478235
Learning Finished
Accuracy: 0.9933
Label:  [0]
Prdiction:  [0]


'\nExtracting MNIST_data/train-images-idx3-ubyte.gz\nExtracting MNIST_data/train-labels-idx1-ubyte.gz\nExtracting MNIST_data/t10k-images-idx3-ubyte.gz\nExtracting MNIST_data/t10k-labels-idx1-ubyte.gz\nLearning started. It takes sometime.\nEpoch: 0001 cost = 0.761663368\nEpoch: 0002 cost = 0.163971262\nEpoch: 0003 cost = 0.112958088\nEpoch: 0004 cost = 0.089053273\nEpoch: 0005 cost = 0.079811646\nEpoch: 0006 cost = 0.069248633\nEpoch: 0007 cost = 0.061179850\nEpoch: 0008 cost = 0.057907291\nEpoch: 0009 cost = 0.053819029\nEpoch: 0010 cost = 0.050987588\nEpoch: 0011 cost = 0.047628223\nEpoch: 0012 cost = 0.043664427\nEpoch: 0013 cost = 0.043872318\nEpoch: 0014 cost = 0.042387685\nEpoch: 0015 cost = 0.039887975\nLearning Finished\nAccuracy: 0.9944\nLabel:  [6]\nPrdiction:  [6]\n'

In [4]:
tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.0015
training_epochs = 15
batch_size = 100

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 784]) # 나중에 변수 만들어줄테니 일단 만들어줘! 이런느낌! (그래프만들어주는느낌?)
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

# 28*28 -> max_pool을 통과하면서 14*14
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # stddev=0.01 : W의 값을 정규화시키겠다는 의미!!
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding="SAME")
L1 = tf.nn.leaky_relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# 14*14 -> max_pool을 통과하면서 7*7
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")
L2 = tf.nn.leaky_relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# 7*7 -> max_pool을 통과하면서 7*7
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding="SAME")
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 4*4*128])

W4 = tf.Variable(tf.random_normal([3, 3, 128, 256], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding="SAME")
L4 = tf.nn.relu(L4)
L4 = tf.nn.avg_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
L4_flat = tf.reshape(L4, [-1, 2*2*256])

# Final FC 7X7X64 inputs -> 10 outputs
W5 = tf.get_variable("W4", shape=[2*2*256,10],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))

# full connected 계산하라!!
logits = tf.matmul(L4_flat, W5)+b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.6} # placeholder에 집어넣겠다는 이야기
        c, _= sess.run([cost, optimizer], feed_dict=feed_dict) # _이라는 말은 optimizer값은 받지 않겠다는 의미
        avg_cost += c/total_batch
        
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))
print("Learning Finished")

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) # argmax : 그 중에 가장 큰거를 1로 하겠다는 의미!!
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={
    X: mnist.test.images, Y: mnist.test.labels, keep_prob:1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples -1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print("Prdiction: ", sess.run(tf.argmax(logits,1), feed_dict={X: mnist.test.images[r:r+1],keep_prob:1}))

sess.close()

"""
Learning started. It takes sometime.
Epoch: 0001 cost = 0.629898750
Epoch: 0002 cost = 0.132082534
Epoch: 0003 cost = 0.095158070
Epoch: 0004 cost = 0.076736899
Epoch: 0005 cost = 0.068833022
Epoch: 0006 cost = 0.060856381
Epoch: 0007 cost = 0.055808209
Epoch: 0008 cost = 0.052807747
Epoch: 0009 cost = 0.050358274
Epoch: 0010 cost = 0.047677156
Epoch: 0011 cost = 0.044894044
Epoch: 0012 cost = 0.044669282
Epoch: 0013 cost = 0.041792488
Epoch: 0014 cost = 0.042411517
Epoch: 0015 cost = 0.042180164
Learning Finished
Accuracy: 0.9945
Label:  [6]
Prdiction:  [6]

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.631447399
Epoch: 0002 cost = 0.131730575
Epoch: 0003 cost = 0.095504604
Epoch: 0004 cost = 0.076603504
Epoch: 0005 cost = 0.067977318
Epoch: 0006 cost = 0.058830276
Epoch: 0007 cost = 0.056941582
Epoch: 0008 cost = 0.052075307
Epoch: 0009 cost = 0.048299915
Epoch: 0010 cost = 0.047720056
Epoch: 0011 cost = 0.047396311
Epoch: 0012 cost = 0.043371768
Epoch: 0013 cost = 0.042666062
Epoch: 0014 cost = 0.041200106
Epoch: 0015 cost = 0.043799160
Learning Finished
Accuracy: 0.9936
Label:  [6]
Prdiction:  [6]
"""

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.631447399
Epoch: 0002 cost = 0.131730575
Epoch: 0003 cost = 0.095504604
Epoch: 0004 cost = 0.076603504
Epoch: 0005 cost = 0.067977318
Epoch: 0006 cost = 0.058830276
Epoch: 0007 cost = 0.056941582
Epoch: 0008 cost = 0.052075307
Epoch: 0009 cost = 0.048299915
Epoch: 0010 cost = 0.047720056
Epoch: 0011 cost = 0.047396311
Epoch: 0012 cost = 0.043371768
Epoch: 0013 cost = 0.042666062
Epoch: 0014 cost = 0.041200106
Epoch: 0015 cost = 0.043799160
Learning Finished
Accuracy: 0.9936
Label:  [6]
Prdiction:  [6]


'\nLearning started. It takes sometime.\nEpoch: 0001 cost = 0.629898750\nEpoch: 0002 cost = 0.132082534\nEpoch: 0003 cost = 0.095158070\nEpoch: 0004 cost = 0.076736899\nEpoch: 0005 cost = 0.068833022\nEpoch: 0006 cost = 0.060856381\nEpoch: 0007 cost = 0.055808209\nEpoch: 0008 cost = 0.052807747\nEpoch: 0009 cost = 0.050358274\nEpoch: 0010 cost = 0.047677156\nEpoch: 0011 cost = 0.044894044\nEpoch: 0012 cost = 0.044669282\nEpoch: 0013 cost = 0.041792488\nEpoch: 0014 cost = 0.042411517\nEpoch: 0015 cost = 0.042180164\nLearning Finished\nAccuracy: 0.9945\nLabel:  [6]\nPrdiction:  [6]\n'

In [4]:
tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.0015
training_epochs = 20
batch_size = 100

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 784]) # 나중에 변수 만들어줄테니 일단 만들어줘! 이런느낌! (그래프만들어주는느낌?)
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

# 28*28 -> max_pool을 통과하면서 14*14
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # stddev=0.01 : W의 값을 정규화시키겠다는 의미!!
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding="SAME")
L1 = tf.nn.leaky_relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# 14*14 -> max_pool을 통과하면서 7*7
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")
L2 = tf.nn.leaky_relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# 7*7 -> max_pool을 통과하면서 7*7
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding="SAME")
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 4*4*128])

W4 = tf.Variable(tf.random_normal([3, 3, 128, 256], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding="SAME")
L4 = tf.nn.relu(L4)
L4 = tf.nn.avg_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
L4_flat = tf.reshape(L4, [-1, 2*2*256])

# Final FC 7X7X64 inputs -> 10 outputs
W5 = tf.get_variable("W4", shape=[2*2*256,10],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))

# full connected 계산하라!!
logits = tf.matmul(L4_flat, W5)+b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.6} # placeholder에 집어넣겠다는 이야기
        c, _= sess.run([cost, optimizer], feed_dict=feed_dict) # _이라는 말은 optimizer값은 받지 않겠다는 의미
        avg_cost += c/total_batch
        
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))
print("Learning Finished")

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) # argmax : 그 중에 가장 큰거를 1로 하겠다는 의미!!
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={
    X: mnist.test.images, Y: mnist.test.labels, keep_prob:1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples -1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print("Prdiction: ", sess.run(tf.argmax(logits,1), feed_dict={X: mnist.test.images[r:r+1],keep_prob:1}))

sess.close()

"""
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.631071149
Epoch: 0002 cost = 0.132339494
Epoch: 0003 cost = 0.095999992
Epoch: 0004 cost = 0.076839354
Epoch: 0005 cost = 0.069623529
Epoch: 0006 cost = 0.060106277
Epoch: 0007 cost = 0.056516412
Epoch: 0008 cost = 0.054262912
Epoch: 0009 cost = 0.048264319
Epoch: 0010 cost = 0.046496579
Epoch: 0011 cost = 0.046426439
Epoch: 0012 cost = 0.044990161
Epoch: 0013 cost = 0.040894123
Epoch: 0014 cost = 0.040263134
Epoch: 0015 cost = 0.043020633
Epoch: 0016 cost = 0.039622556
Epoch: 0017 cost = 0.037282368
Epoch: 0018 cost = 0.036936458
Epoch: 0019 cost = 0.036718981
Epoch: 0020 cost = 0.036856817
Learning Finished
Accuracy: 0.9935
Label:  [9]
Prdiction:  [9]
"""

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.631071149
Epoch: 0002 cost = 0.132339494
Epoch: 0003 cost = 0.095999992
Epoch: 0004 cost = 0.076839354
Epoch: 0005 cost = 0.069623529
Epoch: 0006 cost = 0.060106277
Epoch: 0007 cost = 0.056516412
Epoch: 0008 cost = 0.054262912
Epoch: 0009 cost = 0.048264319
Epoch: 0010 cost = 0.046496579
Epoch: 0011 cost = 0.046426439
Epoch: 0012 cost = 0.044990161
Epoch: 0013 cost = 0.040894123
Epoch: 0014 cost = 0.040263134
Epoch: 0015 cost = 0.043020633
Epoch: 0016 cost = 0.039622556
Epoch: 0017 cost = 0.037282368
Epoch: 0018 cost = 0.036936458
Epoch: 0019 cost = 0.036718981
Epoch: 0020 cost = 0.036856817
Learning Finished
Accuracy: 0.9935
Label:  [9]
Prdiction:  [9]


In [4]:
tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.0015
training_epochs = 15
batch_size = 100

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 784]) # 나중에 변수 만들어줄테니 일단 만들어줘! 이런느낌! (그래프만들어주는느낌?)
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

# 28*28 -> max_pool을 통과하면서 14*14
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # stddev=0.01 : W의 값을 정규화시키겠다는 의미!!
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding="SAME")
L1 = tf.nn.leaky_relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# 14*14 -> max_pool을 통과하면서 7*7
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")
L2 = tf.nn.leaky_relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# 7*7 -> max_pool을 통과하면서 7*7
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding="SAME")
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 4*4*128])

W4 = tf.Variable(tf.random_normal([3, 3, 128, 256], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding="SAME")
L4 = tf.nn.relu(L4)
L4 = tf.nn.avg_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
L4_flat = tf.reshape(L4, [-1, 2*2*256])

# Final FC 7X7X64 inputs -> 10 outputs
W5 = tf.get_variable("W4", shape=[2*2*256,10],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))

# full connected 계산하라!!
logits = tf.matmul(L4_flat, W5)+b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.75} # placeholder에 집어넣겠다는 이야기
        c, _= sess.run([cost, optimizer], feed_dict=feed_dict) # _이라는 말은 optimizer값은 받지 않겠다는 의미
        avg_cost += c/total_batch
        
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))
print("Learning Finished")

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) # argmax : 그 중에 가장 큰거를 1로 하겠다는 의미!!
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={
    X: mnist.test.images, Y: mnist.test.labels, keep_prob:1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples -1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print("Prdiction: ", sess.run(tf.argmax(logits,1), feed_dict={X: mnist.test.images[r:r+1],keep_prob:1}))

sess.close()

"""
Learning started. It takes sometime.
Epoch: 0001 cost = 0.546248281
Epoch: 0002 cost = 0.096679526
Epoch: 0003 cost = 0.066687757
Epoch: 0004 cost = 0.053650952
Epoch: 0005 cost = 0.045954411
Epoch: 0006 cost = 0.040551857
Epoch: 0007 cost = 0.036073856
Epoch: 0008 cost = 0.034324364
Epoch: 0009 cost = 0.029042278
Epoch: 0010 cost = 0.027918621
Epoch: 0011 cost = 0.025404423
Epoch: 0012 cost = 0.025092951
Epoch: 0013 cost = 0.025437099
Epoch: 0014 cost = 0.022435269
Epoch: 0015 cost = 0.021360240
Learning Finished
Accuracy: 0.9937
Label:  [7]
Prdiction:  [7]

Learning started. It takes sometime.
Epoch: 0001 cost = 0.546215439
Epoch: 0002 cost = 0.097268860
Epoch: 0003 cost = 0.066574089
Epoch: 0004 cost = 0.053476030
Epoch: 0005 cost = 0.047204000
Epoch: 0006 cost = 0.040705886
Epoch: 0007 cost = 0.035914054
Epoch: 0008 cost = 0.033133999
Epoch: 0009 cost = 0.030056588
Epoch: 0010 cost = 0.029384942
Epoch: 0011 cost = 0.024879710
Epoch: 0012 cost = 0.025637754
Epoch: 0013 cost = 0.024705053
Epoch: 0014 cost = 0.021834452
Epoch: 0015 cost = 0.020850742
Learning Finished
Accuracy: 0.9933
Label:  [4]
Prdiction:  [4]
"""

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.546215439
Epoch: 0002 cost = 0.097268860
Epoch: 0003 cost = 0.066574089
Epoch: 0004 cost = 0.053476030
Epoch: 0005 cost = 0.047204000
Epoch: 0006 cost = 0.040705886
Epoch: 0007 cost = 0.035914054
Epoch: 0008 cost = 0.033133999
Epoch: 0009 cost = 0.030056588
Epoch: 0010 cost = 0.029384942
Epoch: 0011 cost = 0.024879710
Epoch: 0012 cost = 0.025637754
Epoch: 0013 cost = 0.024705053
Epoch: 0014 cost = 0.021834452
Epoch: 0015 cost = 0.020850742
Learning Finished
Accuracy: 0.9933
Label:  [4]
Prdiction:  [4]


'\nLearning started. It takes sometime.\nEpoch: 0001 cost = 0.546248281\nEpoch: 0002 cost = 0.096679526\nEpoch: 0003 cost = 0.066687757\nEpoch: 0004 cost = 0.053650952\nEpoch: 0005 cost = 0.045954411\nEpoch: 0006 cost = 0.040551857\nEpoch: 0007 cost = 0.036073856\nEpoch: 0008 cost = 0.034324364\nEpoch: 0009 cost = 0.029042278\nEpoch: 0010 cost = 0.027918621\nEpoch: 0011 cost = 0.025404423\nEpoch: 0012 cost = 0.025092951\nEpoch: 0013 cost = 0.025437099\nEpoch: 0014 cost = 0.022435269\nEpoch: 0015 cost = 0.021360240\nLearning Finished\nAccuracy: 0.9937\nLabel:  [7]\nPrdiction:  [7]\n'

In [ ]:
tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.0015
training_epochs = 10
batch_size = 100

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 784]) # 나중에 변수 만들어줄테니 일단 만들어줘! 이런느낌! (그래프만들어주는느낌?)
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

# 28*28 -> max_pool을 통과하면서 14*14
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # stddev=0.01 : W의 값을 정규화시키겠다는 의미!!
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding="SAME")
L1 = tf.nn.leaky_relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# 14*14 -> max_pool을 통과하면서 7*7
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")
L2 = tf.nn.leaky_relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# 7*7 -> max_pool을 통과하면서 7*7
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding="SAME")
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 4*4*128])

W4 = tf.Variable(tf.random_normal([3, 3, 128, 256], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding="SAME")
L4 = tf.nn.relu(L4)
L4 = tf.nn.avg_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
L4_flat = tf.reshape(L4, [-1, 2*2*256])

# Final FC 7X7X64 inputs -> 10 outputs
W5 = tf.get_variable("W4", shape=[2*2*256,10],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))

# full connected 계산하라!!
logits = tf.matmul(L4_flat, W5)+b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.75} # placeholder에 집어넣겠다는 이야기
        c, _= sess.run([cost, optimizer], feed_dict=feed_dict) # _이라는 말은 optimizer값은 받지 않겠다는 의미
        avg_cost += c/total_batch
        
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))
print("Learning Finished")

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) # argmax : 그 중에 가장 큰거를 1로 하겠다는 의미!!
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={
    X: mnist.test.images, Y: mnist.test.labels, keep_prob:1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples -1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print("Prdiction: ", sess.run(tf.argmax(logits,1), feed_dict={X: mnist.test.images[r:r+1],keep_prob:1}))

sess.close()

"""
Learning started. It takes sometime.
Epoch: 0001 cost = 0.546248281
Epoch: 0002 cost = 0.096679526
Epoch: 0003 cost = 0.066687757
Epoch: 0004 cost = 0.053650952
Epoch: 0005 cost = 0.045954411
Epoch: 0006 cost = 0.040551857
Epoch: 0007 cost = 0.036073856
Epoch: 0008 cost = 0.034324364
Epoch: 0009 cost = 0.029042278
Epoch: 0010 cost = 0.027918621
Epoch: 0011 cost = 0.025404423
Epoch: 0012 cost = 0.025092951
Epoch: 0013 cost = 0.025437099
Epoch: 0014 cost = 0.022435269
Epoch: 0015 cost = 0.021360240
Learning Finished
Accuracy: 0.9937
Label:  [7]
Prdiction:  [7]
"""

In [5]:
tf.set_random_seed(777)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.0015
training_epochs = 15
batch_size = 100

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 784]) # 나중에 변수 만들어줄테니 일단 만들어줘! 이런느낌! (그래프만들어주는느낌?)
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

# 28*28 -> max_pool을 통과하면서 14*14
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # stddev=0.01 : W의 값을 정규화시키겠다는 의미!!
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding="SAME")
L1 = tf.nn.leaky_relu(L1)
L1 = tf.nn.avg_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# 14*14 -> max_pool을 통과하면서 7*7
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding="SAME")
L2 = tf.nn.leaky_relu(L2)
L2 = tf.nn.avg_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# 7*7 -> max_pool을 통과하면서 7*7
W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding="SAME")
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3_flat = tf.reshape(L3, [-1, 4*4*128])

W4 = tf.Variable(tf.random_normal([3, 3, 128, 256], stddev=0.01)) # W1이 [3, 3, 1, 32]이므로 32를 가져옴
L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding="SAME")
L4 = tf.nn.relu(L4)
L4 = tf.nn.max_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)
L4_flat = tf.reshape(L4, [-1, 2*2*256])

# Final FC 7X7X64 inputs -> 10 outputs
W5 = tf.get_variable("W4", shape=[2*2*256,10],
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([10]))

# full connected 계산하라!!
logits = tf.matmul(L4_flat, W5)+b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob: 0.75} # placeholder에 집어넣겠다는 이야기
        c, _= sess.run([cost, optimizer], feed_dict=feed_dict) # _이라는 말은 optimizer값은 받지 않겠다는 의미
        avg_cost += c/total_batch
        
    print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.9f}'.format(avg_cost))
print("Learning Finished")

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1)) # argmax : 그 중에 가장 큰거를 1로 하겠다는 의미!!
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={
    X: mnist.test.images, Y: mnist.test.labels, keep_prob:1}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples -1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r+1],1)))
print("Prdiction: ", sess.run(tf.argmax(logits,1), feed_dict={X: mnist.test.images[r:r+1],keep_prob:1}))

sess.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.478477691
Epoch: 0002 cost = 0.106096217
Epoch: 0003 cost = 0.075555537
Epoch: 0004 cost = 0.062084115
Epoch: 0005 cost = 0.052022191
Epoch: 0006 cost = 0.049987101
Epoch: 0007 cost = 0.041778478
Epoch: 0008 cost = 0.039673603
Epoch: 0009 cost = 0.036337151
Epoch: 0010 cost = 0.033507928
Epoch: 0011 cost = 0.032154213
Epoch: 0012 cost = 0.029926537
Epoch: 0013 cost = 0.028983657
Epoch: 0014 cost = 0.025859478
Epoch: 0015 cost = 0.026970431
Learning Finished
Accuracy: 0.9915
Label:  [9]
Prdiction:  [9]
